In [1]:
from collections import defaultdict
import json
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import sys; sys.path.append("models")
from config import get_case_config, get_case_sensitivities
import sensitivity, utils
from strategies import RETURN_KEYS

from chart_books import *

In [2]:
def make_table(entry, key=RETURN_KEYS.reduced_r):
    coords, reslist = zip(*entry)
    vals = take_key(reslist, key)
    mat, idx = utils.sort_by(vals, coords, return_idx=True)
    return pd.DataFrame(np.array(mat).reshape(3, 3))

def config_files(folder):
    return filter(lambda x: x.startswith("config") and x.endswith('.json'), os.listdir(folder))

def run_fname(cfg_fname):
    return cfg_fname.replace("config", "run").replace("json", 'csv')

def resgen(data_dir):
    lockdowns = next(os.walk(data_dir))[1]
    for lockdown in lockdowns:
        folder = os.path.join(data_dir, lockdown)
        for cfg_file in config_files(folder):
            cfg = utils.read_json(os.path.join(folder, cfg_file))
            target = cfg[sensitivity.TARGET_KEY]
            results = pd.read_csv(os.path.join(folder, run_fname(cfg_file)), index_col=0)
            yield lockdown, target, cfg[sensitivity.CONFIG_KEY], results

In [3]:
import utils

keys_to_plot = [RETURN_KEYS.reduced_r, RETURN_KEYS.tests]
rc_dct = {
    'figure.figsize': (14, 6),
    'figure.max_open_warning': 1000,
}
chart_folder = os.path.join(os.environ['REPOS'], 'tti-explorer', 'charts')

pinch_points_dir = os.path.join(os.environ['DATA'], "tti-explorer", "pinch-points")
pinch_points_results = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
for lockdown, target, cfg, results in resgen(pinch_points_dir):
    if target in ['testing_delay', 'manual_trace_delay']:
        continue
    if int(lockdown[1]) > 0:
        level, measures = lockdown.split('_', maxsplit=1)
        pinch_points_results[level][target][measures].append((cfg[target], results))

In [4]:
tt_dir = os.path.join(os.environ['DATA'], "tti-explorer", "pinch-points-test-trace-delay")
test_trace_results = defaultdict(lambda: defaultdict(list))
for lockdown, target, cfg, results in resgen(tt_dir):
    if int(lockdown[1]) > 0:
        vals = [cfg[k] for k in target]
        level, measures = lockdown.split('_', maxsplit=1)
        test_trace_results[level][measures].append((vals, results))  

In [6]:
strings = list()
t1, t2 = ['Test Delay (days)', 'Manual Trace Delay (days)']
table_deck = utils.LatexTableDeck()

for k in sorted(test_trace_results.keys()):
    table_deck.add_section(f"Lockdown Level {k}")
    for policy, entry in test_trace_results[k].items():
        df = make_table(entry)
        df.index =  range(1, 4)
        df.columns = range(1, 4)
        df.index.name = t1
        df.columns.name = t2
        table_deck.add_table(
            tex_table=df.applymap(lambda x: f"{x:.2f}").to_latex(),
            caption=nice_lockdown_name(policy)
        )
    table_deck.clearpage()
table_deck.make("test-trace-tables.tex")


In [29]:
%%capture
with plt.rc_context(rc_dct):
    for level, results in pinch_points_results.items():
        deck = utils.PdfDeck()
        plot_lockdown(results, deck, keys_to_plot)
        deck.make(os.path.join(output_folder, f"{level}_pinch_points.pdf"))

In [30]:
case_sensitivity_dir = os.path.join(os.environ['DATA'], 'tti-explorer', 'case-sensitivity')
for lockdown, target, cfg, results in resgen(case_sensitivity_dir):
    print(lockdown, target)
    

L5_no_contact_tracing 
L2_positive_test_tracing_test_contacts 
L3_symptom_tracing 
L5_positive_test_tracing 
L4_symptom_tracing 
L1_positive_test_tracing_test_contacts 
L4_positive_test_tracing_test_contacts 
L5_symptom_tracing 
L3_positive_test_tracing 
L2_symptom_tracing 
L3_no_contact_tracing 
L1_positive_test_tracing 
L3_positive_test_tracing_test_contacts 
L2_no_contact_tracing 
L1_no_contact_tracing 
L4_positive_test_tracing 
L0 
L5_positive_test_tracing_test_contacts 
L2_positive_test_tracing 
L1_symptom_tracing 
L4_no_contact_tracing 


In [32]:
!ls {os.path.join(case_sensitivity_dir, lockdown)}

config_0.json      over_all_seeds.csv run_0.csv


In [25]:
target

''